In [207]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import openai
from langchain.vectorstores import Pinecone
import os


In [115]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [116]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [117]:
#Function to read docuement
def read_doc(directory):
       dir_loader=PyPDFDirectoryLoader(directory)
       documents=dir_loader.load()
       return(documents)


In [118]:
docs=read_doc("documents/")
len(docs)

43

In [178]:
def chunk_data(docs,chunk_size=500,chunk_overlap=50):
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=50)
        doc=text_splitter.split_documents(docs)
        return doc

In [179]:

chunks=chunk_data(docs)
len(chunks)

234

In [180]:
len(chunks[0].page_content)


500

In [181]:
a=[]
for i in range(len(chunks)):
     print(len(chunks[i].page_content))
     a.append(len(chunks[i].page_content))
     


500
494
497
496
499
225
494
499
498
498
489
497
196
499
491
499
493
493
490
353
494
496
488
499
496
497
126
492
499
493
499
498
217
500
497
496
490
499
493
497
498
494
490
317
500
497
489
493
497
492
286
497
495
497
496
499
320
498
490
493
495
497
193
498
498
493
489
497
218
499
491
489
280
500
497
499
491
404
488
395
498
498
494
491
495
451
499
498
497
498
256
500
499
494
490
209
493
498
499
495
492
399
498
493
499
499
254
500
497
363
497
493
498
496
499
227
495
492
495
498
490
497
497
499
495
498
404
497
496
499
498
186
488
492
492
498
490
453
493
496
496
497
499
64
499
496
496
492
491
495
273
500
496
497
493
498
435
493
497
497
493
493
313
500
496
498
492
498
154
496
498
496
497
79
498
497
495
496
493
494
389
493
493
496
499
497
495
244
499
490
497
498
489
427
496
498
495
492
279
494
499
489
493
495
276
489
493
495
490
498
498
148
495
493
489
482
499
494
492
499
497
293
498
495
497
497
498
389
495
499
402
497
242
157


In [182]:
max(a)

500

In [183]:
len(chunks[0].page_content)

500

In [122]:
embeddings=OpenAIEmbeddings(api_key="")

In [212]:
vec=embeddings.embed_query(chunks[0].page_content)

In [213]:
len(vec)

1536

In [205]:
#Vector searchDb in pinecone
pinecone.init(
    api_key="",
    environment="")
index_name = 'semanticsearchtrial'

In [187]:
Pinecone.

In [216]:
import time
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
        pinecone.create_index(        
        name=index_name,
        dimension=1536,
        metric='cosine'

    )
    # wait a moment for the index to be fully initialized
time.sleep(1)

In [217]:

index=Pinecone.from_documents(chunks,embeddings,index_name=index_name)

In [218]:
##Cosine similarity to retrive results
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k)
    return matching_results


In [223]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
llm=OpenAI(model_name="text_davinci_003",temperature=0.5,api_key="")
chain=load_qa_chain(llm,chain_type="stuff")
